In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from commit_transformer.tokenizer import build_vocab
from commit_transformer.preprosser import CommitDataset
from commit_transformer.model import CombinedModel

In [2]:
df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df

train, test = train_test_split(df, test_size=0.3, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)
train,_ = train_test_split(train, train_size=0.1, random_state=42)
test, val = train_test_split(test, train_size=0.1, random_state=42)
val, _ = train_test_split(val, train_size=0.1, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

/tmp/ipykernel_1797/2474789719.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


In [3]:
train_data = []
for index, row in train.iterrows():
  train_data.append([row['commit_message'],row['diff_code'],row['category']])

val_data = []
for index, row in val.iterrows():
  val_data.append([row['commit_message'],row['diff_code'],row['category']])
test_data = []
for index, row in test.iterrows():
  test_data.append([row['commit_message'],row['diff_code'],row['category']])

In [4]:

# # Split the dataset
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# # Build vocabulary
vocab = build_vocab(train_data)

# Create Datasets and DataLoaders
train_dataset = CommitDataset(train_data, vocab)
val_dataset = CommitDataset(val_data, vocab)
test_dataset = CommitDataset(test_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [5]:
model = torch.load("transformer_entire_bert_model.pth")

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report

In [7]:
test_labels, test_predictions = model.evaluate(test_loader)
print(classification_report(test_labels,test_predictions,digits=4))

Validation Accuracy: 0.7374301675977654
Precision: 0.6326530612244898
Recall: 0.3712574850299401
F1-Score: 0.4679245283018868
              precision    recall  f1-score   support

           0     0.7608    0.9027    0.8257       370
           1     0.6327    0.3713    0.4679       167

    accuracy                         0.7374       537
   macro avg     0.6967    0.6370    0.6468       537
weighted avg     0.7210    0.7374    0.7144       537

